In [6]:
import numpy as np
import pandas as pd
import re
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import sklearn as skl    

In [7]:
def My_preprocesing(url):
    dataset_df = pd.read_csv(url)
    # Wyrzucanie niepotrzbnych rzeczy 
    dataset_df = dataset_df.drop(["zipcode","thumbnail_url","id","neighbourhood","description","name","host_response_rate"],axis = 1)
    
    change_names = dataset_df.property_type.value_counts().index.tolist()[5:]
    for index,data in dataset_df.iterrows():
        if data[1] in change_names:
            dataset_df.property_type[index] = 'Other'
    
    #one hot encoding
    ammenities_set = set()
    for index,data in dataset_df.iterrows():
        temp = data[3].replace('{', ',')
        temp1 =temp.replace('}', ',')
        temp2 =  re.findall('(?<=,)[^,]+(?=,)', temp1)
        for i in temp2:
            ammenities_set.add(i)
    z = np.zeros((dataset_df.shape[0],len(ammenities_set)))
    ammenities_df = pd.DataFrame(z,columns=ammenities_set)
    arr =ammenities_df.values
    ammenities_list = list(ammenities_set)
    for index,data in dataset_df.iterrows():
        temp = data[3].replace('{', ',')
        temp =temp.replace('}', ',')
        temp =  re.findall('(?<=,)[^,]+(?=,)', temp)
        for fraze in temp:
            arr[index,ammenities_list.index(fraze)] = 1
    ammenities_df = pd.DataFrame(arr,columns=ammenities_set)
    suma = []
    for i in ammenities_list:
        suma.append(ammenities_df[i].sum())
    get_rid = []
    for i in range(len(suma)):
        if suma[i] < 10000: # patrząc po wykresie sumy wydawało się to dobrym pomysłem
            get_rid.append(ammenities_list[i])
    ammenities_df=  ammenities_df.drop(get_rid,axis=1)
    
    
    def encoding(lablename,data):
        le = LabelEncoder()
        le.fit( data[lablename])
        data[lablename]= le.transform(data[lablename].values)
        return data
    
    arr = dataset_df.bed_type.values
    arr[arr != 'Real Bed'] = 'Other'
    dataset_df.bed_type = arr
    col_enc = ["room_type","city","bed_type","host_has_profile_pic", "host_identity_verified","instant_bookable","cancellation_policy","cleaning_fee","property_type"]
    for i in col_enc:
        dataset_df = encoding(i,dataset_df)
    dataset_df= dataset_df.drop(["amenities"],axis=1)
    
    
        
    def time_coding(columns,dataset_df):
        for i in columns:
            dataset_df[i] = dataset_df[i].astype('datetime64[ns]')
            min = dataset_df[i].min()
            temp = []
            for idx in range(dataset_df.shape[0]):
                if math.isnan((dataset_df[i][idx] - min).days):
                    temp.append(0)
                else:
                    temp.append(int((dataset_df[i][idx] - min).days))
            dataset_df[i] = pd.DataFrame(temp)
            
            
        return dataset_df
    
    dataset_df =time_coding(["host_since","last_review","first_review"],dataset_df)
    
    for i in ["host_since","last_review","first_review"]:
        n = i
        scaler = skl.preprocessing.StandardScaler()
        scaler.fit(dataset_df[n].values.reshape(-1,1))
        dataset_df[n] = scaler.transform(dataset_df[n].values.reshape(-1,1)) 
        
    return pd.concat([dataset_df, ammenities_df], axis=1, join='inner').dropna()
    

In [8]:
data = My_preprocesing("train.csv")

C:\Users\macie\AppData\Local\Temp/ipykernel_15756/92403983.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_df.property_type[index] = 'Other'


In [9]:
Y,X = data['log_price'].values,data.drop(['log_price'],axis=1).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.33, random_state=42)

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier


In [14]:
Ridge1 = skl.linear_model.Ridge()
RFR = RandomForestRegressor(n_estimators = 150, random_state = 0)
clf = RandomForestClassifier()

In [12]:
from S_regresor import Stacker_Regresion

In [15]:
my_stacker = Stacker_Regresion([Ridge1,RFR],Classifier=clf)

In [16]:
my_stacker.fit(X=X_train,Y=Y_train)

training model, name: RandomForestRegressor(n_estimators=150, random_state=0)


d:\Nauka\Polibuda\V Semestr\ML\Kontaminacja\S_regresor.py:61: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  hist.append( Stacker_Regresion.Classfier.fit(X_train2,classes))


[Ridge(),
 RandomForestRegressor(n_estimators=150, random_state=0),
 RandomForestClassifier()]

In [17]:
my_stacker.evaluate_models(X=X_test,Y=Y_test)

UnboundLocalError: local variable 'i' referenced before assignment